In [1]:

from explainers.random_forest import RFExplainer
from explainers.linear_regression import LRExplainer
from explainers.linear_regression_noconstrain import LRnoconExplainer
from explainers.utils import compute_log_odds

from explainers.kernel_shap import KernelShapExplainer
from explainers.kernel_shap_1constrain import KernelShapExplainer1constraint

from data.mask_windows import WindowMaskingDataGenerator
from data.base_generator import MaskingConfig

from models.ast.model import ASTModel
from models.cough.model import CoughModel
from models.drums.model import DrumsModel
from models.kws.model import KWSModel

import numpy as np
from utils import get_segments
import os
import json
from pathlib import Path
from data.random_generator import RandomDataGenerator


/opt/conda/envs/tf-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-16 19:42:39.636509: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 19:42:39.671103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739734959.720854 2942496 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739734959.736702 2942496 cuda_blas.cc:1418] Unable 

In [5]:
config = {"stat": {"mask_percentage": [0.2, 0.3, 0.4], "possible_windows": [1,3,5], "possible_mask_types": ['stat'], "combinations": 9}}
filename= 'mnt/data/drum_dataset/5019.wav'
random_data = RandomDataGenerator(
                path='/home/ec2-user/results1/explanations_drums', 
                model_name='drums',
                filename=filename,
                config=config,
                num_samples=3000, 
            )

In [ ]:
model_name = 'drums'
id_to_explain = 5
filename = f'mnt/data/drum_dataset/5019.wav'

if model_name == 'drums':
    model = DrumsModel(filename, id_to_explain) 
    complete_filename = filename
    filename = os.path.basename(filename)

input, real_score_id = model.process_input()     
predict_fn = model.get_predict_fn()   
pred_zeros = predict_fn([np.zeros(len(input))])
if model_name == 'drums':
    empty_score = compute_log_odds(pred_zeros, id_to_explain)[0]
    print(empty_score)
else:
    empty_score = pred_zeros[id_to_explain]

In [3]:
name = 'stat'
        
kernelshap_analyzer = KernelShapExplainer(
    path= Path(path) / filename / model_name / f"scores_{name}.json",
)
importances_kernelshap = kernelshap_analyzer.explain_instance(
    label_to_explain=id_to_explain, empty_score=empty_score, model=model_name).get_feature_importances()

kernelshap_analyzer_1constraint = KernelShapExplainer1constraint(
    path= Path(path) / filename / model_name / f"scores_{name}.json",
)
importances_kernelshap_analyzer_1constraint = kernelshap_analyzer_1constraint.explain_instance(
    label_to_explain=id_to_explain, empty_score=empty_score, model=model_name).get_feature_importances()

rf_analyzer = RFExplainer(
    path= Path(path) / filename / model_name / f"scores_{name}.json",
    filename=filename,
)
importances_rf_tree = rf_analyzer.get_feature_importances(label_to_explain=id_to_explain, method='tree', model=model_name)

# LR analysis
lime_nocon_analyzer = LRnoconExplainer(
    Path(path) / filename / model_name / f"scores_{name}.json",
    verbose=False,
    absolute_feature_sort=False
)
importances_nocon = lime_nocon_analyzer.explain_instance(
    label_to_explain=id_to_explain, model=model_name).get_feature_importances()

lime_analyzer = LRExplainer(
    Path(path) / filename / model_name / f"scores_{name}.json",
    verbose=False,
    absolute_feature_sort=False
)
importances_lime = lime_analyzer.explain_instance(
    label_to_explain=id_to_explain, model=model_name).get_feature_importances()

true_markers = get_segments(complete_filename, id_to_explain, model_name)
############## Prepare output ##############
output_data = {
    "metadata": {
        "filename": filename,
        "id_explained": float(id_to_explain),
        "segment_length": 100,
        "true_markers": true_markers,
        "true_score": real_score_id
    },
    "importance_scores": {
        "importances_kernelshap_analyzer_1constraint": {
            "method": "NaiveAudioAnalyzer",
            "values": importances_kernelshap_analyzer_1constraint.tolist() if hasattr(importances_kernelshap_analyzer_1constraint, 'tolist') else importances_kernelshap_analyzer_1constraint
        },
        "random_forest_tree_importance": {
                "method": "masked rf with tree importance",
                "values": importances_rf_tree.tolist() if hasattr(importances_rf_tree, 'tolist') else importances_rf_tree
        },
        "linear_regression": {
            "method": "Linear Regression with kernel as pi",
            "values": importances_lime.tolist() if hasattr(importances_lime, 'tolist') else importances_lime
        },
        "linear_regression_nocon": {
            "method": "Linear Regression with kernel as pi",
            "values": importances_nocon.tolist() if hasattr(importances_nocon, 'tolist') else importances_nocon
        },
        "kernel_shap": {
            "method": "Linear Regression with shap as pi",
            "values": importances_kernelshap.tolist() if hasattr(importances_kernelshap, 'tolist') else importances_kernelshap  
        }
    }
}
print(output_data["importance_scores"]["importances_kernelshap_analyzer_1constraint"]["values"])